In [1]:
import pandas as pd
import datetime as dt
import gradio as gr


In [2]:
import warnings 
pd.options.mode.chained_assignment = None # se desactiva advertencias de pandas por la edición_sobrescritura

In [3]:
# Lectura manual
data=pd.read_csv("../data/Automated_Traffic_Volume_Counts.csv")
air_quality=pd.read_csv("../data/finale_dataset/air_pollution.csv")

## Calidad de aire en PM 2,5

Se calcula el PM 2,5 particulado promedio de cada borough de todos los años de análisis

In [4]:
air_quality=air_quality[air_quality['polluting_agent'] == 'PM 2.5 (µg/cm3)'].reset_index()


In [5]:
air_quality.groupby(['borough_name'])['data_value'].mean().reset_index()

,borough_name,data_value
0,bronx,9.709606
1,brooklyn,9.263040
2,manhattan,10.874167
3,queens,8.929167
4,staten island,8.486759


## Volumen de trafico 

Se calcula el volumen de trafico de todos los años de análisis para cada borough

In [6]:
# Se renombra columnas

data = data.rename(columns={'Yr': 'year', 'M': 'month','D': 'day' })

# Se crea columna 'date'

data['date'] = pd.to_datetime(data[['year', 'month', 'day']])



In [7]:
# Se toman las variables necesarias

data_new=data[['Boro','date','Vol','WktGeom']]

# Se renombra columnas

data_new.columns=['borough', 'date', 'volume', 'WktGeom']

# se establece rango desde 2010 al 2020
data_new=data_new.loc[data['date'] > '2010-01-01']

# Se modifica la columna 'Date' en formato año

data_new['date']=data_new['date'].dt.year

In [8]:
# Se agrupa por la posiccion geografica con la media para calcular el volumen de trafico

data_new=data_new.groupby(['WktGeom', 'date', 'borough'])['volume'].mean().reset_index()

# Se agrupa por año y por borough

data_new=data_new.groupby(['date', 'borough'])['volume'].sum().reset_index().sort_values('volume', ascending=False)
data_new.groupby('borough')['volume'].mean().reset_index().sort_values('volume', ascending=False)

,borough,volume
2,Manhattan,58420.411266
3,Queens,54046.743319
1,Brooklyn,49682.108176
0,Bronx,31839.043899
4,Staten Island,8962.459457


De la contaminación al aire por PM 2,5 en NYC el sector de tráfico genera el 17%.<br>
Del sector de tráfico los carros generan un 5%, y los camiones 12%. 

https://a816-dohbesp.nyc.gov/IndicatorPublic/Traffic/index.html

In [9]:
# Funcion para calcular el % de disminucion de contaminación al aire por PM 2,5  en el sector de automoviles 

def kpi_brooklyn(carros):
    
    volume_brooklyn = 58420 * 4 * 24
    pm_brooklyn = 9.26
    pm_brooklyn=pm_brooklyn*0.05
   
    new_volume_brooklyn = (volume_brooklyn-(carros*4*24))
    new_pm_brooklyn= (new_volume_brooklyn*pm_brooklyn)/volume_brooklyn
    ptc_disminuye = (new_pm_brooklyn - pm_brooklyn)/pm_brooklyn
    brooklyn=round(-100*ptc_disminuye,2)
    
    return brooklyn

    
def kpi_manhattan(carros):
    
    volume_manhattan = 49682 * 4 * 24
    pm_manhattan = 11
    pm_manhattan=pm_manhattan*0.05
   
    new_volume_manhattan = (volume_manhattan-(carros*4*24))
    new_pm_manhattan= (new_volume_manhattan*pm_manhattan)/volume_manhattan
    ptc_disminuye = (new_pm_manhattan - pm_manhattan)/pm_manhattan
    manhattan=round(-100*ptc_disminuye,2)
    
    return manhattan
    

def kpi_queen(carros):
    volume_queen = 54046 * 4 * 24
    pm_queen = 9
    pm_queen=pm_queen*0.05
   
    new_volume_queen = (volume_queen-(carros*4*24))
    new_pm_queen= (new_volume_queen*pm_queen)/volume_queen
    ptc_disminuye = (new_pm_queen - pm_queen)/pm_queen
    queen=round(-100*ptc_disminuye,2)
    
       
    return queen

def kpi_staten_island(carros):
    volume_staten_island = 8962 * 4 * 24
    pm_staten_island = 8.5
    pm_staten_island=pm_staten_island*0.05
   
    new_volume_staten_island = (volume_staten_island-(carros*4*24))
    new_pm_staten_island= (new_volume_staten_island*pm_staten_island)/volume_staten_island
    ptc_disminuye = (new_pm_staten_island - pm_staten_island)/pm_staten_island
    staten_island=round(-100*ptc_disminuye,2)
    
    return staten_island

def kpi_the_bronx(carros):
    
    volume_the_bronx = 31839 * 4 * 24
    pm_the_bronx = 9.7
    pm_the_bronx=pm_the_bronx*0.05
   
    new_volume_bronx = (volume_the_bronx-(carros*4*24))
    new_pm_the_bronx= (new_volume_bronx*pm_the_bronx)/volume_the_bronx
    ptc_disminuye = (new_pm_the_bronx - pm_the_bronx)/pm_the_bronx
    the_bronx=round(-100*ptc_disminuye,2)
    
    return the_bronx

def taxis(carros):
    
    volume_taxi = 7737
    pm_nyc = 0.47
  
   
    new_volume_taxi = (volume_taxi-carros)
    new_pm_nyc= (new_volume_taxi*pm_nyc)/volume_taxi
    ptc_disminuye = (new_pm_nyc - pm_nyc)/pm_nyc
    taxis=round(-100*ptc_disminuye,2)
    
    return taxis
    

def disminucion_sector_automoviles(carros):
    kpi_b = kpi_brooklyn(carros)
    kpi_m = kpi_manhattan(carros)
    kpi_q = kpi_queen(carros)
    kpi_s = kpi_staten_island(carros)
    kpi_bx= kpi_the_bronx(carros)
    kpi_taxis= taxis(carros)
    
    indicador=f"{int(carros)} carros eléctricos disminuyen la contaminación al aire por PM 2,5 generada por el sector de los automoviles en: \n {kpi_b}% en Brooklyn. \n {kpi_m}% en Manhattan. \n {kpi_q}% en Quenns. \n {kpi_s}% Staten en Islands \n {kpi_bx}% en Bronx. \n \n{int(carros)} taxis eléctricos disminuyen un {kpi_taxis}% de la contaminacón al aire por PM 2,5 generada por el sector de taxis en NYC"
    return indicador

In [10]:
disminucion_sector_automoviles(7000)

'7000 carros eléctricos disminuyen la contaminación al aire por PM 2,5 generada por el sector de los automoviles en: \n 11.98% en Brooklyn. \n 14.09% en Manhattan. \n 12.95% en Quenns. \n 78.11% Staten en Islands \n 21.99% en Bronx. \n \n7000 taxis eléctricos disminuyen un 90.47% de la contaminacón al aire por PM 2,5 generada por el sector de taxis en NYC'

In [12]:
import gradio as gr

title = "REPORTE MEDIOAMBIENTAL"
table_html = """
   <table>
  <tr>
    <th>borough</th>
    <th>Pm 2,5</th>
  </tr>
  <tr>
    <td>bronx</td>
    <td>9.7</td>
  </tr>
  <tr>
    <td>brooklyn</td>
    <td>9.3</td>
  </tr>
  <tr>
    <td>manhattan</td>
    <td>10.9</td>
  </tr>
  <tr>
    <td>queens</td>
    <td>8.9</td>
  </tr>
  <tr>
    <td>staten island</td>
    <td>8.5</td>
  </tr>
</table>

"""

with gr.Blocks(title=title) as demo:
    text = gr.components.HTML("""
        <h1>Indicadores Ambientales</h1>
        <h2>Contaminación al aire</h2>
        
        <p> La contaminación al aire por PM 2.5 Promedio al año en los distintos distritos de New York es:
        
        
        """ + table_html + """
        
        
        El sector de transporte genera el 17% de la contaminacón. <br>
        
        <img src="https://drive.google.com/file/d/1ZOI97Wi5lc73SylyLg71JHLDlul9bX8P/view?usp=share_link" alt="descripción de la imagen">


            
       Se genera un indicador clave de rendimiento que estipula el Porcentaje de disminución de la contaminacón al aire por el uso 
       de carros electricos.<br>

       <b> KPI: Valor Porcentual de disminución de la contaminación al aire de PM 2,5 por # de Autos Eléctricos </b>
        
        </p>
    """)
    
    vehiculos_electricos = gr.Number(label="Numero de Vehículos Electricos en la Flota")
    calcular_btn = gr.Button("% disminución a la contaminación al aire per # Vehículos Eléctricos:")
    indicador = gr.Textbox(label = "Indicadores:")
    
    calcular_btn.click(fn=disminucion_sector_automoviles, inputs=[vehiculos_electricos], outputs=[indicador])
    

demo.launch(share=True)



Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://248cbe862164dc2c09.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
